In [57]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
import csv
import pandas as pd
import numpy as np
import sklearn

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

MAX_PIP_COUNT = 40
NUM_COLORS = 5
NUM_IN_PACK = 13
NUM_PACKS = 3

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    columns = next(draft_data)

names = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
PICK_IDX = columns.index('pick')
DRAFTID_IDX = columns.index('draft_id')
DRAFTTIME_IDX = columns.index('draft_time')
PACKNUM_IDX = columns.index('pack_number')
PICKNUM_IDX = columns.index('pick_number')
PACK_IDX = columns.index('pack_card_A Killer Among Us')
POOL_IDX = columns.index('pool_A Killer Among Us')
NUM_CARDS = len(names)


In [5]:
def print_pack(row):
    print( "========" )
    print( f"Draft Id: {row[DRAFTID_IDX]}, Pack {row[PACKNUM_IDX]}, Pick {row[PICKNUM_IDX]}")
    print( "--------" )
    pack_vector = card_vector(PACK_IDX, row)
    if (pack_vector.sum() > 1):
        pack_alsa = np.where(pack_vector, alsa, NUM_IN_PACK + 1)
        min_indices = np.argpartition(pack_alsa, 2)
        for idx in min_indices[0:2]:
            print( names[idx] + f" (ALSA: {pack_alsa[idx]:.2f})" )
        print( f"And {pack_vector.sum() - 2} other cards" )
    else:
        print( names[pack_vector.argmax()] )
    print("--------")
    print(f"Picked: {row[PICK_IDX]}")
    print("========")
    print("")


def card_vector(index, row):
    return np.array([int(x) for x in row[index:index + NUM_CARDS]])

In [55]:
alsa_filepath = './data/alsa.npy'
alsa = np.load(alsa_filepath)

In [56]:
alsa

array([2.91717717, 4.7545917 , 4.50481194, 5.92018383, 5.1366312 ,
       1.50945626, 5.45776339, 5.26211109, 1.96938776, 3.30701123,
       4.96212531, 1.96921444, 2.98049263, 3.64336508, 3.4277919 ,
       4.61438577, 1.20962199, 1.45652174, 4.20273646, 1.75788468,
       2.78811572, 5.116219  , 6.03648233, 5.69045883, 5.41588785,
       3.66676463, 3.55045492, 3.57722284, 4.90474302, 5.61999221,
       5.03012972, 5.45644866, 4.78904963, 2.87748883, 4.35244771,
       5.35338124, 4.62046923, 3.18094438, 2.9728072 , 4.19191813,
       2.30652842, 5.21768111, 2.69719388, 4.37134503, 4.97653283,
       4.86827185, 2.41829362, 3.70668949, 4.52062642, 5.57169293,
       5.02598327, 5.84854018, 4.63063502, 4.2129477 , 3.47154544,
       3.76635469, 4.54803543, 3.10621996, 4.52966109, 3.92153285,
       4.54057481, 3.30123495, 5.06778165, 4.05279537, 3.1217906 ,
       1.87043189, 4.57336971, 4.20415597, 4.14486871, 4.59578078,
       1.50533808, 4.12215312, 4.68908893, 4.98600618, 4.29531

In [8]:
pool_when_taken = np.zeros((len(names), len(names)), dtype = np.intc)
taken_counts = np.zeros((len(names)))

skipped_count = 0

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    next(draft_data)
    row = next(draft_data)
    
    while(row[DRAFTTIME_IDX] < '2024-02-20'):
        skipped_count += 1
        row = next(draft_data)
            
    while(row and row[DRAFTTIME_IDX] < '2024-04-01'):
        pick = names.index(row[PICK_IDX])
        pool_vector = card_vector(POOL_IDX, row)
        pool_when_taken[pick] += pool_vector
        taken_counts[pick] += 1
        try:
            row = next(draft_data)
        except:
            row = None


In [44]:
ihd_df = pd.read_csv('ihd_df_6apr24.csv', index_col=0)
gpwr = np.array(ihd_df.loc['gpwr'])

In [46]:
pool_quality = gpwr - np.matmul(pool_when_taken, gpwr.reshape(len(names),1)).reshape(len(names),) / pool_when_taken.sum(axis=1)

/tmp/ipykernel_30021/4168655802.py:1: RuntimeWarning: invalid value encountered in divide
  pool_quality = gpwr - np.matmul(pool_when_taken, gpwr.reshape(len(names),1)).reshape(len(names),) / pool_when_taken.sum(axis=1)


In [53]:
names[np.nan_to_num(pool_quality, 10).argsort()]

TypeError: only integer scalar arrays can be converted to a scalar index

In [40]:
pool_when_taken.sum(axis=1).reshape(1

array([216771, 139869, 203536, 436699, 164086,  30002, 486760, 397904,
        28159,  77157,  77386,  27305,  68463,  50765,  46980,  74371,
        29455,  59160, 481568,  62241,  34961, 389201, 483956, 472615,
        12824, 484647,  56212, 131797, 178602, 181658, 498388,  13915,
       153548, 184777, 144107, 168071, 162921, 141086,  55106, 151954,
       148204, 146390,  67718, 128022,  77485, 188078,  50675, 133388,
        58677, 177254, 185428, 400109, 204718, 127447, 183953, 414984,
        15253,  68088, 155792,  58508,  15088,  20447, 170343, 146347,
        60496,   5489, 135053, 401437, 445549,  70388,  63795, 166442,
       173952, 134927, 130115,  53218, 448967, 408052,  22795, 424303,
       174509, 451201,  49492,  84572,  62135, 392814,   4585, 419865,
        17458,  62314, 196343,  17791, 493487, 148778,  28185, 213977,
         5682, 163623,  34584, 171828, 405407,  57014,   5501, 157258,
       439129, 548506, 473910, 177513,   4886, 169862, 179654,  53444,
      

In [54]:
row